In [1]:
import os
from pyhdf.SD import SD, SDC
from shapely.geometry import Polygon
from pyproj import Transformer

In [2]:
def get_files(dir_ = '../data/MODIS_MOD13Q1'):
    return [
        os.path.join(dir_, f) for f in os.listdir(dir_) if f.endswith('hdf')
    ]

In [3]:
files = get_files()

In [4]:
files

['../data/MODIS_MOD13Q1/MOD13Q1.A2022353.h13v09.061.2023006060213.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023049.h13v09.061.2023070133537.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023113.h13v09.061.2023130000837.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023081.h13v09.061.2023100005743.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023129.h13v09.061.2023146131839.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023033.h13v09.061.2023054033015.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023065.h13v09.061.2023082002406.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023001.h13v09.061.2023019193552.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023017.h13v09.061.2023034172156.hdf',
 '../data/MODIS_MOD13Q1/MOD13Q1.A2023097.h13v09.061.2023115120734.hdf']

In [5]:
file = files[0]
file

'../data/MODIS_MOD13Q1/MOD13Q1.A2022353.h13v09.061.2023006060213.hdf'

In [6]:
hdf = SD(file, SDC.READ)

In [7]:
hdf.datasets()

{'250m 16 days NDVI': (('YDim:MODIS_Grid_16DAY_250m_500m_VI',
   'XDim:MODIS_Grid_16DAY_250m_500m_VI'),
  (4800, 4800),
  22,
  0),
 '250m 16 days EVI': (('YDim:MODIS_Grid_16DAY_250m_500m_VI',
   'XDim:MODIS_Grid_16DAY_250m_500m_VI'),
  (4800, 4800),
  22,
  1),
 '250m 16 days VI Quality': (('YDim:MODIS_Grid_16DAY_250m_500m_VI',
   'XDim:MODIS_Grid_16DAY_250m_500m_VI'),
  (4800, 4800),
  23,
  2),
 '250m 16 days red reflectance': (('YDim:MODIS_Grid_16DAY_250m_500m_VI',
   'XDim:MODIS_Grid_16DAY_250m_500m_VI'),
  (4800, 4800),
  22,
  3),
 '250m 16 days NIR reflectance': (('YDim:MODIS_Grid_16DAY_250m_500m_VI',
   'XDim:MODIS_Grid_16DAY_250m_500m_VI'),
  (4800, 4800),
  22,
  4),
 '250m 16 days blue reflectance': (('YDim:MODIS_Grid_16DAY_250m_500m_VI',
   'XDim:MODIS_Grid_16DAY_250m_500m_VI'),
  (4800, 4800),
  22,
  5),
 '250m 16 days MIR reflectance': (('YDim:MODIS_Grid_16DAY_250m_500m_VI',
   'XDim:MODIS_Grid_16DAY_250m_500m_VI'),
  (4800, 4800),
  22,
  6),
 '250m 16 days view zenith

In [8]:
DATAFIELD_NAME='250m 16 days NDVI'
data3D = hdf.select(DATAFIELD_NAME)

In [9]:
data3D.attributes()

{'long_name': '250m 16 days NDVI',
 'units': 'NDVI',
 'valid_range': [-2000, 10000],
 '_FillValue': -3000,
 'scale_factor': 10000.0,
 'scale_factor_err': 0.0,
 'add_offset': 0.0,
 'add_offset_err': 0.0,
 'calibrated_nt': 5}

In [10]:
data3D.info()

('250m 16 days NDVI', 2, [4800, 4800], 22, 9)

In [11]:
hdf.attributes().keys()

dict_keys(['HDFEOSVersion', 'StructMetadata.0', 'identifier_product_doi', 'identifier_product_doi_authority', 'CoreMetadata.0', 'ArchiveMetadata.0'])

In [12]:
import ast

def parse_structure(text):
    lines = text.strip().split("\n")
    stack = [{}]

    for line in lines:
        line = line.strip()
        if line.startswith("GROUP="):
            group_name = line.split("=")[1]
            new_group = {}
            if group_name in stack[-1]:
                if isinstance(stack[-1][group_name], list):
                    stack[-1][group_name].append(new_group)
                else:
                    stack[-1][group_name] = [stack[-1][group_name], new_group]
            else:
                stack[-1][group_name] = new_group
            stack.append(new_group)
        elif line.startswith("END_GROUP="):
            stack.pop()
        elif line.startswith("OBJECT="):
            object_name = line.split("=")[1]
            new_object = {}
            if object_name in stack[-1]:
                if isinstance(stack[-1][object_name], list):
                    stack[-1][object_name].append(new_object)
                else:
                    stack[-1][object_name] = [stack[-1][object_name], new_object]
            else:
                stack[-1][object_name] = new_object
            stack.append(new_object)
        elif line.startswith("END_OBJECT="):
            stack.pop()
        else:
            if "=" in line:
                key, value = line.split("=", 1)
                key = key.strip()
                value = value.strip()

                # Handle tuples (lists of numbers) or lists of strings
                if value.startswith("(") and value.endswith(")"):
                    value_content = value[1:-1]
                    try:
                        # Try parsing as a tuple of numbers
                        value = tuple(map(float, value_content.split(',')))
                    except ValueError:
                        # If fails, treat it as a list of strings
                        value = ast.literal_eval(f'[{value_content}]')
                elif value.isdigit():
                    value = int(value)
                elif value.startswith('"') and value.endswith('"'):
                    value = value.strip('"')

                stack[-1][key] = value

    return stack[0]

In [13]:
text=hdf.attributes()["StructMetadata.0"]

In [14]:
print(text)

GROUP=SwathStructure
END_GROUP=SwathStructure
GROUP=GridStructure
	GROUP=GRID_1
		GridName="MODIS_Grid_16DAY_250m_500m_VI"
		XDim=4800
		YDim=4800
		UpperLeftPointMtrs=(-5559752.598333,0.000000)
		LowerRightMtrs=(-4447802.078667,-1111950.519667)
		Projection=GCTP_SNSOID
		ProjParams=(6371007.181000,0,0,0,0,0,0,0,0,0,0,0,0)
		SphereCode=-1
		GROUP=Dimension
		END_GROUP=Dimension
		GROUP=DataField
			OBJECT=DataField_1
				DataFieldName="250m 16 days NDVI"
				DataType=DFNT_INT16
				DimList=("YDim","XDim")
			END_OBJECT=DataField_1
			OBJECT=DataField_2
				DataFieldName="250m 16 days EVI"
				DataType=DFNT_INT16
				DimList=("YDim","XDim")
			END_OBJECT=DataField_2
			OBJECT=DataField_3
				DataFieldName="250m 16 days VI Quality"
				DataType=DFNT_UINT16
				DimList=("YDim","XDim")
			END_OBJECT=DataField_3
			OBJECT=DataField_4
				DataFieldName="250m 16 days red reflectance"
				DataType=DFNT_INT16
				DimList=("YDim","XDim")
			END_OBJECT=DataField_4
			OBJECT=DataField_5
				DataFi

In [15]:
result = parse_structure(text)

In [16]:
import pyproj
from pyproj import Proj, Transformer

def convert_to_epsg4326(parsed_dict):
    grid_structure = parsed_dict.get("GridStructure", {}).get("GRID_1", {})
    
    upper_left = grid_structure.get("UpperLeftPointMtrs")
    lower_right = grid_structure.get("LowerRightMtrs")
    proj_params = grid_structure.get("ProjParams")
    
    if not upper_left or not lower_right or not proj_params:
        raise ValueError("Missing necessary projection parameters.")
    
    # Define the MODIS Sinusoidal Projection
    sinusoidal_proj = Proj("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
    
    # Define transformation to WGS84 (EPSG:4326)
    transformer = Transformer.from_proj(sinusoidal_proj, "epsg:4326", always_xy=True)
    
    # Convert coordinates
    west, north = transformer.transform(upper_left[0], upper_left[1])
    east, south = transformer.transform(lower_right[0], lower_right[1])
    
    return (west, north, east, south)

In [17]:
bounding_box = convert_to_epsg4326(result)

In [ ]:
def process_array(array, fill_value, scale_factor):
    array = array.astype(float)  # Ensure the array is float to support NaN
    array[array == fill_value] = np.nan  # Replace fill values with NaN
    array *= scale_factor  # Scale the remaining values
    return array

In [18]:
data3D.get()

array([[ 5373,  5373,  5270, ..., -3000, -3000, -3000],
       [ 4593,  5468,  5468, ..., -3000, -3000, -3000],
       [ 4008,  5468,  5468, ..., -3000, -3000, -3000],
       ...,
       [ 8134,  8200,  7697, ...,  7647,  7715,  7715],
       [ 9127,  9127,  4241, ...,  7604,  7518,  7518],
       [ 9628,  9739,  4866, ...,  7502,  7663,  7663]],
      shape=(4800, 4800), dtype=int16)

In [19]:
bounding_box

(-49.999999995506855, 0.0, -40.61706447167344, -9.999999999104968)

In [20]:
from shapely.geometry import Polygon
import pandas as pd

def map_matrix_to_dataframe(matrix, bounding_box, polygon_to_check=None):
    rows, cols = matrix.shape

    # Bounding box coordinates
    west, north, east, south = bounding_box

    # Compute pixel size
    pixel_width = (east - west) / cols
    pixel_height = (north - south) / rows

    data = []

    for i in range(rows):
        for j in range(cols):
            # Compute pixel bounding box
            min_x = west + j * pixel_width
            max_x = min_x + pixel_width
            min_y = north - (i + 1) * pixel_height
            max_y = min_y + pixel_height

            # Create polygon for the pixel
            polygon = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y), (min_x, min_y)])
            
            # If polygon_to_check is provided, check for intersection
            if polygon_to_check is None or polygon.intersects(polygon_to_check):
                # Store in list if it intersects or no polygon to check
                data.append((matrix[i, j], polygon.wkt))

    # Create DataFrame
    df = pd.DataFrame(data, columns=["NDVI", "wkt"])
    return df


In [21]:
from shapely import wkt

In [22]:
polygon_to_check = wkt.loads(
    "POLYGON((-48.837 -1.83, -47.6921 -1.83, -47.6921 -3.3766, -48.837 -3.3766, -48.837 -1.83))"
)

In [23]:
matrix = data3D.get()
output = map_matrix_to_dataframe(matrix, bounding_box, polygon_to_check=polygon_to_check)

In [24]:
output.to_csv('output.csv')